Script para buscar resumos das teses e dissertações na BDTD

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import numpy as np
import pandas as pd
import json

In [2]:
# setup url

takeoff = '8857' #Pedra Bonita
page = '1'
url = ('http://www.xcbrasil.com.br/tracks/world/alltimes/takeoff:'+takeoff+'&sortOrder=DATE&page_num='+page)

In [3]:
# Request webpage and parser html file
f = requests.get(url).text 
soup = bs(f, "html.parser")

In [74]:
for row in soup.find_all('tr', {"class": ["l_row1", "l_row2"]}):
    data = []
    # 'id', 'indexcell', 'date', 'pilot/local', 'country', 'duration', 'straight_line_distance', 
    #'olc_distance', 'olc_pontuation', 'track_class', 'none', 'equipment', 'none2', 'brand', 'none3',  'flightLink'
    
    # id
    data.append(row['id'])
    
    # Get all html tags
    col = row.find_all('td')
    
    #date
    data.append(col[1].text.strip())
    
    #pilot
    data.append(col[2].text.split('\n')[0].strip())
    
    #local
    data.append(col[2].text.split('\n')[1].strip())

    #country
    data.append(col[2].find('img')['title'])
    
    #duration
    data.append(col[3].text)
    
    #straight_line_distance(km)
    data.append(col[4].text.split('\xa0')[0])
    
    #olc_distance
    data.append(col[5].text.split('\xa0')[0])
    
    #olc_pontuation
    data.append(col[6].text.strip())

    #track_class
    data.append(col[6].find('img')['title'].strip())
    
    #equipment
    data.append(col[7].find('img')['title'].strip(' -'))
    
    #brands
    data.append(col[8].find('img')['title'].strip())
    #print(col[8])
    
    #flightLink
    data.append('http://www.xcbrasil.com.br/'+ col[9].find('a', {"class": "flightLink"})['href'])
    

['row_492378', '23/02/2021', 'Dennis Feld', 'Pedra Bonita - BR', 'Brasil', '1:02', '8.2', '18.9', '33.00', 'Triângulo livre', 'Parapente - Open', 'Flow XCracer M', 'http://www.xcbrasil.com.br//flight/492378']
['row_492399', '23/02/2021', 'Gustavo Alves Russi', 'Pedra Bonita - BR', 'Brasil', '1:01', '8.1', '16.2', '32.45', 'Triângulo FAI', 'Parapente - Open', 'Flow Xc Racer Ml', 'http://www.xcbrasil.com.br//flight/492399']
['row_492403', '23/02/2021', 'Marcelo Araripe', 'Pedra Bonita - BR', 'Brasil', '1:02', '8.2', '17.4', '30.37', 'Triângulo livre', 'Parapente - Open', 'Ozone Zeno ML', 'http://www.xcbrasil.com.br//flight/492403']
['row_492392', '23/02/2021', 'Phil Haegler', 'Pedra Bonita - BR', 'Brasil', '0:37', '6.4', '9.2', '13.73', 'Distância livre', 'Parapente - Sport', 'Skywalk Chili4 S', 'http://www.xcbrasil.com.br//flight/492392']
['row_492401', '23/02/2021', 'Reginaldo Pirozi', 'Pedra Bonita - BR', 'Brasil', '0:49', '6.1', '7.5', '11.29', 'Distância livre', 'Parapente - Sport',

In [19]:
    
    #    try:
            data.append(col.text)
        except:
            pass

        try:        
            if col.div['class'] == ['pilotLink']:
                data.append(col.find('img')['title'])
        except:
            pass
        
        try:
            if col['class'] == ['OLCScore']:
                data.append(col.find('img')['title'])
        except:
            pass
        
        try:        
            if col.div['class'] == ['catInfo']:
                data.append(col.find('img')['title'])
        except:
            pass
        
        
        try:        
            if 'brands' in col.find('img')['class']:
                data.append(col.find('img')['title'])
        except:
            pass
        
        try:
            data.append('http://www.xcbrasil.com.br/'+ col.find('a', {"class": "flightLink"})['href'])
        except:
            pass
        

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 3)

In [75]:
#Coletando link para as teses

#df = pd.DataFrame(columns=['id', 'indexcell', 'date', 'pilot/local', 'country', 'duration', 'straight_line_distance', 
# 'olc_distance', 'olc_pontuation', 'track_class', 'none', 'equipment', 'none2', 'brand', 'mostrar', 'flightLink'])

datas = []

for row in soup.find_all('tr', {"class": ["l_row1", "l_row2"]}):
    
    data = []
    
     # id
    data.append(row['id'])
    
    # Get all html tags
    col = row.find_all('td')
    
    #date
    data.append(col[1].text.strip())
    
    #pilot
    data.append(col[2].text.split('\n')[0].strip())
    
    #local
    data.append(col[2].text.split('\n')[1].strip())

    #country
    data.append(col[2].find('img')['title'])
    
    #duration
    data.append(col[3].text)
    
    #straight_line_distance(km)
    data.append(col[4].text.split('\xa0')[0])
    
    #olc_distance(km)
    data.append(col[5].text.split('\xa0')[0])
    
    #olc_pontuation
    data.append(col[6].text.strip())

    #track_class
    data.append(col[6].find('img')['title'].strip())
    
    #equipment
    data.append(col[7].find('img')['title'].strip(' -'))
    
    #brands
    data.append(col[8].find('img')['title'].strip())
    #print(col[8])
    
    #flightLink
    data.append('http://www.xcbrasil.com.br/'+ col[9].find('a', {"class": "flightLink"})['href'])
    
        
    datas.append(data)

datas[0]
df = pd.DataFrame(data = datas, columns=['id', 'date', 'pilot', 'local', 'country', 'duration', 
                                         'straight_line_distance(km)', 'olc_distance(km)', 'olc_pontuation', 
                                         'track_class', 'equipment', 'brand', 'flightLink'])
df

,id,date,pilot,local,country,duration,straight_line_distance(km),olc_distance(km),olc_pontuation,track_class,equipment,brand,flightLink
0,row_492378,23/02/2021,Dennis Feld,Pedra Bonita - BR,Brasil,1:02,8.2,18.9,33.00,Triângulo livre,Parapente - Open,Flow XCracer M,http://www.xcbrasil.com.br//flight/492378
1,row_492399,23/02/2021,Gustavo Alves Russi,Pedra Bonita - BR,Brasil,1:01,8.1,16.2,32.45,Triângulo FAI,Parapente - Open,Flow Xc Racer Ml,http://www.xcbrasil.com.br//flight/492399
2,row_492403,23/02/2021,Marcelo Araripe,Pedra Bonita - BR,Brasil,1:02,8.2,17.4,30.37,Triângulo livre,Parapente - Open,Ozone Zeno ML,http://www.xcbrasil.com.br//flight/492403
3,row_492392,23/02/2021,Phil Haegler,Pedra Bonita - BR,Brasil,0:37,6.4,9.2,13.73,Distância livre,Parapente - Sport,Skywalk Chili4 S,http://www.xcbrasil.com.br//flight/492392
4,row_492401,23/02/2021,Reginaldo Pirozi,Pedra Bonita - BR,Brasil,0:49,6.1,7.5,11.29,Distância livre,Parapente - Sport,SOL Ciclus1,http://www.xcbrasil.com.br//flight/492401
5,row_492571,23/02/2021,Thiago Cotta,Pedra Bonita - BR,Brasil,0:35,1.7,4.8,9.65,Triângulo FAI,Asa delta FAI1 - Kingpost,WillsWing Falcon 140,http://www.xcbrasil.com.br//flight/492571
6,row_492411,23/02/2021,Mauricio Simoes,Pedra Bonita - BR,Brasil,0:24,1.9,5.0,7.50,Distância livre,Parapente - Sport,Skywalk Chili4 S,http://www.xcbrasil.com.br//flight/492411
7,row_492408,23/02/2021,Gustavo Lenine Velasco de Oliveira,Pedra Bonita - BR,Brasil,1:33,1.7,4.3,6.49,Distância livre,Parapente - Sport,Apco VISTA3 L,http://www.xcbrasil.com.br//flight/492408
8,row_492544,23/02/2021,Andre Malicia,Pedra Bonita - BR,Brasil,0:31,1.8,3.9,5.92,Distância livre,Parapente - Sport,SOL SycrossOne L,http://www.xcbrasil.com.br//flight/492544
9,row_492370,23/02/2021,flavio Battioli Coimbra,Pedra Bonita - BR,Brasil,0:37,1.6,2.4,3.56,Distância livre,,SOL Ellus 2,http://www.xcbrasil.com.br//flight/492370


In [29]:
areas = pd.read_csv('Areas_CNPQ.csv', sep=';')

In [30]:
#função para coletar link para cada tese de uma determinada áreas do conhecimento do CNPQ

def link_area(area, page):
    #Separar nome e sobrenome do autor
    nivel_1 = area[0]
    nivel_2 = area[1]

    #Preparar nome e sobrenome para a url
    nivel_1 = '+'.join(nivel_1.split())
   
    if str(nivel_2) != 'nan':
        nivel_2 = '+'.join(nivel_2.split())
        
        #preparar a url
        url = ('http://bdtd.ibict.br/vufind/Search/Results?filter%5B%5D=dc.subject.cnpq.fl_str_mv%3A"CNPQ%3A%3A' +
               nivel_1 +
               '%3A%3A' +
               nivel_2 +
               '"&page=' +
               str(page))
    else:
        url = ('http://bdtd.ibict.br/vufind/Search/Results?filter%5B%5D=dc.subject.cnpq.fl_str_mv%3A"CNPQ%3A%3A' +
               nivel_1 +
               '"&page=' +
               str(page))
    
    #Fazer requisição e parsear o arquivo html
    f = requests.get(url, proxies = proxies).text 
    soup = bs(f, "html.parser")

    #Coletando link para as teses
    links = []
    for doc in soup.find_all('a', href=True):
        if 'title' in doc.get('class', []):
            links.append(doc['href'])
    return links

In [31]:
#Coletar o link com as teses das áreas não relacionadas à Petrobras
area_oposta = areas[areas['PETROBRAS'] == 0]

n_pages = 10 # Cada página retorna 20 teses
links_area_oposta = []

for area in area_oposta.iterrows():
    for p in range(n_pages):
        link = link_area(area[1], p)
        if link != []:
            links_area_oposta = links_area_oposta + link
        else:
            break

In [32]:
#Coletar o link com as teses das áreas não relacionadas à Petrobras
mesma_area = areas[areas['PETROBRAS'] == 1]

n_pages = 10 # Cada página retorna 20 teses
links_mesma_area = []

for area in mesma_area.iterrows():
    for p in range(n_pages):
        link = link_area(area[1], p)
        if link != []:
            links_mesma_area = links_mesma_area + link
        else:
            break

In [33]:
# Função para coletar os metadados de uma tese ou dissertação da BDTD dado uma URL
def tese_link(url):
    #definir url
    url = 'http://bdtd.ibict.br' + link
    
    #Requisitar html e fazer o parser
    f = requests.get(url, proxies = proxies).text 
    soup = bs(f, "html.parser")

    #Dicionário para armazenar as informações da tese
    tese = {}  
    
    #Adicionar título
    tese['Title'] = soup.find('h3').get_text()
    for doc in soup.find_all('tr'):
       
        #Identificar atributo
        try:
            atributo = doc.find('th').get_text()
        except:
            pass
        
        #Verificar se o atributo possui mais de um dado
        for row in doc.find_all('td'):
            
            #Adicionar o atributo no diconário
            if row.find('div') == None:
                try:
                    tese[atributo] = doc.find('td').get_text()
                except:
                    pass
            else:
                element = []
                
                #No dicionário, adicionar todos os dados ao seu respectivo atributo
                for e in doc.find_all('div'):
                    try:
                        element.append(e.find('a').get_text())
                    except:
                        pass
                tese[atributo] = element
    
    return(tese)

In [38]:
teses = {}
n = 0
for link in links_area_oposta:
    url = 'http://bdtd.ibict.br'+link
    teses[url] = tese_link(link)
    n = n + 1
    if n % 500 == 0:
        print(n)
        with open('teses_areas_opostas_Large.json', 'w') as fp:
            json.dump(teses, fp)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500


In [39]:
teses = {}
n = 0
for link in links_mesma_area:
    url = 'http://bdtd.ibict.br'+link
    teses[url] = tese_link(link)
    n = n + 1
    if n % 500 == 0:
        print(n)
        with open('teses_mesmas_areas_Large.json', 'w') as fp:
            json.dump(teses, fp)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
